# Named Entity Recognition with TensorFlow and 🔭 Galileo

In this tutorial, we'll train a model with Tensorflow and explore the results in Galileo.

**Make sure to select GPU in your Runtime! (Runtime -> Change Runtime type)**

In [ ]:
#@title Install `dataquality`

# A small function for minimizing the dataset for testing
import os

def _minimize_for_ci() -> bool:
    return os.getenv("MINIMIZE_FOR_CI", "false") == "true"

    
try:
    import dataquality as dq
except ImportError:
    # Upgrade pip
    !pip install -U pip &> /dev/null

    # Install HF datasets for downloading the example datasets
    !pip install -U dataquality datasets transformers &> /dev/null
    
    print('👋 Installed necessary libraries and restarting runtime! This should only need to happen once.')
    print('🙏 Continue with the rest of the notebook or hit "Run All" again!')

    # Restart the runtime
    import os, time
    time.sleep(1) # gives the print statements time to flush
    os._exit(0) # exits without allowing the next cell to run

# 1. Login to Galileo

In [ ]:
import dataquality as dq

dq.login()

# 2. Load Data

In [ ]:
#@title 🤗 HuggingFace Dataset
#@markdown You can select any dataset from [here](https://huggingface.co/datasets?language=language:en&task_categories=task_categories:token-classification&task_ids=task_ids:named-entity-recognition&sort=downloads) which contains train/validation/test splits and an `ner_tags` column.

dataset_name = 'conllpp' #@param ["wnut_17", "conllpp", "wikiann"] {allow-input: true}
print(f"You selected the {dataset_name} dataset")

from IPython.utils import io
from datasets import load_dataset, get_dataset_config_names

# Try to load the data. If a config (subset) is needed, pick one
try:
  with io.capture_output() as captured:
    data = load_dataset(dataset_name)
except ValueError as e:
  if "Config name is missing" not in repr(e):
    raise e

  configs = get_dataset_config_names(dataset_name)
  print(f"The dataset {dataset_name} has multiple subsets {configs}.")
  config = input(f"🖖 Enter the name of the subset to pick (or leave blank for any): ")
  if config:
    assert config in configs, f"{config} is not a valid subset"
  else:
    config = configs[0]
  with io.capture_output() as captured:
    data = load_dataset(dataset_name, name=config)

# Check that the dataset has at least train and either of validation/test
assert {"train", "validation", "test"}.issubset(data), \
f"💾 The dataset {dataset_name} has either no train, validation or test splits, select another one."

print(f"\n🏆 Dataset {dataset_name} loaded succesfully")

# 3. Initialize Galileo

In [ ]:
# 🔭🌕 Initializing a new run in Galileo. Each run goes under a project.
dq.init(task_type="text_ner", 
        project_name="named_entity_recognition_tensorflow", 
        run_name=f"example_run_{dataset_name.replace('/', '-')}")

# 4. Tokenize and Log Dataset
Galileo provides HF integrations to allow tokenization, and label alignment, while automatically logging your input data. Next, we prepare a dataset loader for each split.  

In [ ]:
import pandas as pd
from transformers import DataCollatorForTokenClassification
from dataquality.integrations import hf
from transformers import AutoTokenizer
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorForTokenClassification(tokenizer, return_tensors="tf")

MINIBATCH_SIZE = 32
NUM_EPOCHS = 5

# 🔭🌕 Galileo tokenizes the HuggingFace DatasetDict logs the dataset(s) present in it
tokenized_datasets = hf.tokenize_and_log_dataset(data, tokenizer)

# Train and test datasets
tf_train_set = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "id"],
    shuffle=True,
    batch_size=MINIBATCH_SIZE,
    collate_fn=data_collator,
)

tf_validation_set = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "id"],
    shuffle=False,
    batch_size=MINIBATCH_SIZE,
    collate_fn=data_collator,
)

tf_test_set = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "id"],
    shuffle=False,
    batch_size=MINIBATCH_SIZE,
    collate_fn=data_collator,
)

# 5. Prepare and Log NER Model 

Model data can be logged via `log_model_outputs`. This step will log the model logits and embeddings. You can achieve this by adding 1 line of code to any NER model. 

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForTokenClassification, TFAutoModel

class TokenClassificationModel(tf.keras.Model):
  """Example TF Model-subclass-style TokenClassificationModel model

  Internally uses a huggingface TF model for the feature extractor
  """

  def __init__(self, num_labels):
      super().__init__()
      self.feature_extractor = TFAutoModel.from_pretrained("distilbert-base-uncased")
      self.classifier = tf.keras.layers.Dense(num_labels)

  def call(self, x, ids):
      input_ids, attention_ids = x

      encoded_layers = self.feature_extractor(input_ids, attention_ids).last_hidden_state
      logits = self.classifier(encoded_layers)

      # 🔭🌕 Log the model's logits and embeddings
      dq.log_model_outputs(
          embs=encoded_layers[:, 1:, :],
          logits=logits[:, 1:, :],
          ids=ids,
      )

      return logits

# 6. Putting into Action: Training a Model

Complete the training pipeline using a standard TensorFlow training setup. While training, log the current `epoch` and `split`.

Call `dq.finish()` after training to complete logging.

In [ ]:
from tqdm.notebook import tqdm
from transformers.modeling_tf_utils import shape_list

def compute_loss(labels, logits):
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction=tf.keras.losses.Reduction.NONE
  )
  # make sure only labels that are not equal to -100
  # are taken into account as loss
  active_loss = tf.reshape(labels, (-1,)) != -100
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
  return loss_fn(labels, reduced_logits)

NUM_EPOCHS = 5

num_labels = len(data[list(data)[0]].features["ner_tags"].feature.names)
model = TokenClassificationModel(num_labels=num_labels)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

# Sparse means label can be provided as class idx, not one hot
train_metric = tf.metrics.SparseCategoricalAccuracy()
test_metric = tf.metrics.SparseCategoricalAccuracy()

# Iterate over epochs.
for epoch in range(NUM_EPOCHS):
  print("Start of epoch %d" % (epoch,))
  dq.set_epoch(epoch) # 🔭🌕 Galileo logging
  train_metric.reset_states()
  test_metric.reset_states()

  # Iterate over the batches of the train dataset.
  dq.set_split("training") # 🔭🌕 Galileo logging
  for train_batch in tqdm(tf_train_set):
      y = train_batch["labels"]
      x = (train_batch["input_ids"], train_batch["attention_mask"])

      with tf.GradientTape() as tape:
          logits = model(x,ids=train_batch["id"])
          loss = compute_loss(y, logits)

      train_metric.update_state(y, logits)
      grads = tape.gradient(loss, model.trainable_weights)
      optimizer.apply_gradients(zip(grads, model.trainable_weights))

      if _minimize_for_ci():
        break


  # Iterate over the batches of the test dataset.
  dq.set_split("test") # 🔭🌕 Galileo logging
  for test_batch in tqdm(tf_test_set):
      y = test_batch["labels"]
      x = (test_batch["input_ids"], test_batch["attention_mask"])

      logits = model(x, ids=test_batch["id"])
      loss = compute_loss(y, logits)
      test_metric.update_state(y, logits)
      
      if _minimize_for_ci():
        break

dq.finish() # 🔭🌕 Galileo logging

# General Help and Docs
- To get help with your task's requirements, call `dq.get_data_logger().doc()`
- To see more general data and model logging docs, run `dq.docs()`

In [ ]:
dq.get_data_logger().doc()
help(dq.log_dataset)